In [29]:
from selenium import webdriver

from pymongo import MongoClient
import re
import pandas as pd
from bs4 import BeautifulSoup
import json

In [9]:
player = {
    'id': 1628389,
    'stats': {}
}
browser = webdriver.Chrome(executable_path="./chromedriver")
browser.get(f'https://stats.nba.com/player/203518/?SeasonType=Regular%20Season')
# Get player name and format it for the folder name
name = browser.find_elements_by_xpath('//div[@class="player-summary__player-name"]')
position = browser.find_elements_by_xpath('//span[@ng-if="playerInfo.POSITION_INITIALS"]')
player['name'] = ' '.join(name[0].text.split('\n'))
player['position'] = position[0].text if len(position) > 0 else ''
# Find the table where all the statistics tables are
stats_tables = browser.find_elements_by_xpath('//nba-stat-table')
# Iterate over each table and create your own csv file
for index_table, stats_table in enumerate(stats_tables):
    # Get the table name
    caption_item = browser.find_elements_by_xpath('.//div[@class="nba-stat-table__caption"]')
    caption = 'Table'+str(index_table) if len(caption_item) == 0 else caption_item[index_table].text
    player['stats'][caption] = []
    # Get the columns names
    head = []
    col_list = stats_table.find_elements_by_xpath('.//div[@class="nba-stat-table__overflow"]/table/thead/tr/th')
    # Get the values of each row
    row_list = stats_table.find_elements_by_xpath('.//div[@class="nba-stat-table__overflow"]/table/tbody/tr')
    for index_row, row in enumerate(row_list):
        body = {}
        body_col_list = row.find_elements_by_xpath('.//td')
        for index_col, row_col in enumerate(body_col_list):
            body[col_list[index_col].text] = row_col.text
        player['stats'][caption].append(body)
print(player)

{'id': 1628389, 'stats': {'Traditional Splits': [{'BY YEAR': '2018-19', 'TEAM': 'OKC', 'GP': '31', 'MIN': '19.0', 'PTS': '5.3', 'FGM': '1.8', 'FGA': '5.1', 'FG%': '35.7', '3PM': '1.3', '3PA': '4.1', '3P%': '32.3', 'FTM': '0.4', 'FTA': '0.4', 'FT%': '92.3', 'OREB': '0.2', 'DREB': '1.4', 'REB': '1.5', 'AST': '0.6', 'TOV': '0.5', 'STL': '0.5', 'BLK': '0.2', 'PF': '1.7', 'FP': '9.9', 'DD2': '0', 'TD3': '0', '+/-': '0.9'}, {'BY YEAR': '2017-18', 'TEAM': 'OKC', 'GP': '75', 'MIN': '15.1', 'PTS': '4.7', 'FGM': '1.5', 'FGA': '3.9', 'FG%': '39.5', '3PM': '1.1', '3PA': '2.9', '3P%': '38.0', 'FTM': '0.5', 'FTA': '0.6', 'FT%': '84.8', 'OREB': '0.3', 'DREB': '1.2', 'REB': '1.5', 'AST': '0.4', 'TOV': '0.3', 'STL': '0.5', 'BLK': '0.1', 'PF': '1.7', 'FP': '8.6', 'DD2': '0', 'TD3': '0', '+/-': '0.6'}, {'BY YEAR': '2016-17', 'TEAM': 'OKC', 'GP': '68', 'MIN': '15.5', 'PTS': '6.0', 'FGM': '2.0', 'FGA': '5.0', 'FG%': '39.3', '3PM': '1.4', '3PA': '3.6', '3P%': '38.1', 'FTM': '0.6', 'FTA': '0.7', 'FT%': '89.8

In [34]:
player = {
    'id': 1628389,
    'stats': {}
}
browser = webdriver.Chrome(executable_path="./chromedriver")
browser.get(f'https://stats.nba.com/player/1629061/?SeasonType=Regular%20Season')
# Get player name and format it for the folder name
name = browser.find_elements_by_xpath('//div[@class="player-summary__player-name"]')
position = browser.find_elements_by_xpath('//span[@ng-if="playerInfo.POSITION_INITIALS"]')
player['name'] = ' '.join(name[0].text.split('\n'))
player['position'] = position[0].text if len(position) > 0 else ''

caption_item = browser.find_elements_by_xpath('.//div[@class="nba-stat-table__caption"]')

soup = BeautifulSoup(browser.page_source, 'lxml')
tables = soup.findAll('nba-stat-table')

for index, table in enumerate(tables):
    df = pd.read_html(table.prettify())

    print(caption_item[index].text)
    print(df[0].head())
    
    json_stats = df[0].to_dict(orient='records')
    
    print(json_stats)

Traditional Splits
   By Year TEAM  GP   MIN  PTS  FGM  FGA   FG%  3PM  3PA  ...  REB  AST  TOV  \
0  2018-19  CLE  19  10.2  1.7  0.6  1.9  30.6  0.3  1.2  ...  1.0  0.3  0.3   

   STL  BLK   PF   FP  DD2  TD3  +/-  
0  0.1  0.2  0.7  3.8    0    0 -1.6  

[1 rows x 26 columns]
[{'By Year': '2018-19', 'TEAM': 'CLE', 'GP': 19, 'MIN': 10.2, 'PTS': 1.7, 'FGM': 0.6, 'FGA': 1.9, 'FG%': 30.6, '3PM': 0.3, '3PA': 1.2, '3P%': 26.1, 'FTM': 0.2, 'FTA': 0.2, 'FT%': 100, 'OREB': 0.2, 'DREB': 0.8, 'REB': 1.0, 'AST': 0.3, 'TOV': 0.3, 'STL': 0.1, 'BLK': 0.2, 'PF': 0.7, 'FP': 3.8, 'DD2': 0, 'TD3': 0, '+/-': -1.6}]
Advanced Splits
   By Year TEAM  GP   MIN  OffRtg  DefRtg  NetRtg  AST%  AST/TO  AST Ratio  \
0  2018-19  CLE  19  10.2   100.0   109.5    -9.5   3.8    0.83       10.2   

   OREB%  DREB%  REB%  TO Ratio  eFG%   TS%  USG%   PACE  PIE  
0    1.4    8.9   4.8      12.2  38.9  42.4   9.7  96.66  1.8  
[{'By Year': '2018-19', 'TEAM': 'CLE', 'GP': 19, 'MIN': 10.2, 'OffRtg': 100.0, 'DefRtg': 109